# Neural Network (PyTorch)

- A **custom feedforward neural network** (`FraudNN`) was built using PyTorch, with:
  - Two hidden layers (64 and 32 units)
  - `ReLU` activations and a `Dropout` layer (p=0.3)
  - A `Sigmoid` output for binary classification

- The dataset was **standardized** using `StandardScaler`.

- **Class imbalance** was addressed by passing a custom `pos_weight` to `BCELoss`, calculated as the ratio of legitimate to fraud cases.

- The model was trained for **50 epochs** using the **Adam optimizer** with a learning rate of 0.001.

- A **custom threshold of 0.7** was applied to the sigmoid output during evaluation to reduce false positives.

- Final performance was measured using a confusion matrix and classification report.

In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from torch.utils.data import DataLoader, TensorDataset

# Load and preprocess the data
df = pd.read_csv("data/creditcard_isoforest_cleaned_001.csv")
X = df.drop("Class", axis=1)
y = df["Class"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_legit = X_scaled[y == 0]

# Split test set for evaluation
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

# Torch datasets
train_loader = DataLoader(TensorDataset(torch.tensor(X_legit, dtype=torch.float32)), batch_size=2048, shuffle=True)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.int)

# Define the Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 20),
            nn.ReLU(),
            nn.Linear(20, 10)
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Model setup
device = torch.device("cpu")  # або "cuda" якщо доступний
model = Autoencoder(X.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Train the model
EPOCHS = 200
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        x_batch = batch[0].to(device)
        optimizer.zero_grad()
        output = model(x_batch)
        loss = criterion(output, x_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss:.4f}")

# Compute reconstruction error
model.eval()
with torch.no_grad():
    reconstructions = model(X_test_tensor.to(device))
    mse = torch.mean((X_test_tensor - reconstructions.cpu()) ** 2, dim=1)

    # Threshold (mean + 3*std from legit data)
    recon_legit = model(torch.tensor(X_legit, dtype=torch.float32).to(device))
    recon_mse = torch.mean((torch.tensor(X_legit, dtype=torch.float32) - recon_legit.cpu()) ** 2, dim=1)
    threshold = recon_mse.mean() + 3 * recon_mse.std()

# Classify
y_pred = (mse > threshold).int()

# Evaluate
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, digits=4, target_names=["Legit", "Fraud"]))

# Save model with auto-incremented filename
import os

model_dir = "models"
os.makedirs(model_dir, exist_ok=True)
base_filename = "fraud_nn_tuned"
ext = ".pt"

i = 0
while True:
    filename = f"{base_filename}{'' if i == 0 else f'_{i:02d}'}{ext}"
    filepath = os.path.join(model_dir, filename)
    if not os.path.exists(filepath):
        break
    i += 1

torch.save(model.state_dict(), filepath)
print(f"Model saved to {filepath}")


Epoch 1/200, Loss: 120.5741
Epoch 2/200, Loss: 86.3876
Epoch 3/200, Loss: 72.7200
Epoch 4/200, Loss: 65.9608
Epoch 5/200, Loss: 61.4218
Epoch 6/200, Loss: 58.3176
Epoch 7/200, Loss: 56.1627
Epoch 8/200, Loss: 54.5278
Epoch 9/200, Loss: 53.2985
Epoch 10/200, Loss: 52.2352
Epoch 11/200, Loss: 51.2572
Epoch 12/200, Loss: 50.3834
Epoch 13/200, Loss: 49.3685
Epoch 14/200, Loss: 48.5354
Epoch 15/200, Loss: 47.6297
Epoch 16/200, Loss: 46.9201
Epoch 17/200, Loss: 46.3157
Epoch 18/200, Loss: 45.7583
Epoch 19/200, Loss: 45.3787
Epoch 20/200, Loss: 45.0509
Epoch 21/200, Loss: 44.8148
Epoch 22/200, Loss: 44.5398
Epoch 23/200, Loss: 44.3951
Epoch 24/200, Loss: 44.1865
Epoch 25/200, Loss: 44.0802
Epoch 26/200, Loss: 43.9777
Epoch 27/200, Loss: 43.8595
Epoch 28/200, Loss: 43.7905
Epoch 29/200, Loss: 43.8246
Epoch 30/200, Loss: 43.6651
Epoch 31/200, Loss: 43.5809
Epoch 32/200, Loss: 43.4714
Epoch 33/200, Loss: 43.4535
Epoch 34/200, Loss: 43.3542
Epoch 35/200, Loss: 43.4051
Epoch 36/200, Loss: 43.3448


1. **Model Update**  
   - A **deeper Autoencoder** was implemented to better capture complex patterns in legitimate transaction data.  
   - Architecture includes:
     - Encoder: `input → 128 → 64 → 32 → 16`
     - Decoder: `16 → 32 → 64 → 128 → input`

2. **Training Configuration**  
   - Trained on only legitimate transactions (`Class == 0`) using MSE loss.  
   - Optimizer: Adam with a reduced learning rate (`1e-4`) for more stable convergence.  
   - Epochs: 50

3. **Reconstruction Threshold**  
   - A new threshold was calculated using the **mean + 3×std** of reconstruction error on legit data (based on the deeper model’s output).  
   - Used to classify anomalies on the full test set.

4. **Model Saving**  
   - The model was saved under an incremented name format to avoid overwriting earlier versions.

In [ ]:
# Torch datasets
train_loader = DataLoader(TensorDataset(torch.tensor(X_legit, dtype=torch.float32)), batch_size=2048, shuffle=True)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.int)

# 4. Define a deeper Autoencoder
class DeepAutoencoder(nn.Module):
    def __init__(self, input_dim):
        super(DeepAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16)
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

# Setup model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepAutoencoder(X.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Train
EPOCHS = 50
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in train_loader:
        x_batch = batch[0].to(device)
        optimizer.zero_grad()
        output = model(x_batch)
        loss = criterion(output, x_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss:.4f}")

# Compute reconstruction error & threshold
model.eval()
with torch.no_grad():
    reconstructions = model(X_test_tensor.to(device))
    mse = torch.mean((X_test_tensor - reconstructions.cpu()) ** 2, dim=1)

    # Compute threshold based on training (legit only)
    recon_legit = model(torch.tensor(X_legit, dtype=torch.float32).to(device))
    recon_mse = torch.mean((torch.tensor(X_legit, dtype=torch.float32) - recon_legit.cpu()) ** 2, dim=1)
    threshold = recon_mse.mean() + 3 * recon_mse.std()

# Predict
y_pred = (mse > threshold).int()

# Evaluate
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, digits=4, target_names=["Legit", "Fraud"]))

# Save model (auto-increment)
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)
base_filename = "fraud_nn_tuned"
ext = ".pt"
i = 1
while True:
    filename = f"{base_filename}_{i:02d}{ext}"
    filepath = os.path.join(model_dir, filename)
    if not os.path.exists(filepath):
        break
    i += 1

torch.save(model.state_dict(), filepath)
print(f"Model saved to {filepath}")

Epoch 1/50, Loss: 123.1475
Epoch 2/50, Loss: 94.8220
Epoch 3/50, Loss: 81.7899
Epoch 4/50, Loss: 72.6615
Epoch 5/50, Loss: 67.4970
Epoch 6/50, Loss: 64.3542
Epoch 7/50, Loss: 61.9401
Epoch 8/50, Loss: 59.6646
Epoch 9/50, Loss: 57.8871
Epoch 10/50, Loss: 56.4318
Epoch 11/50, Loss: 55.3271
Epoch 12/50, Loss: 54.3253
Epoch 13/50, Loss: 53.3518
Epoch 14/50, Loss: 52.5826
Epoch 15/50, Loss: 51.8236
Epoch 16/50, Loss: 51.1989
Epoch 17/50, Loss: 50.5586
Epoch 18/50, Loss: 49.8553
Epoch 19/50, Loss: 49.3822
Epoch 20/50, Loss: 48.7197
Epoch 21/50, Loss: 48.2306
Epoch 22/50, Loss: 47.5477
Epoch 23/50, Loss: 46.8588
Epoch 24/50, Loss: 46.1776
Epoch 25/50, Loss: 45.5948
Epoch 26/50, Loss: 45.0252
Epoch 27/50, Loss: 44.5524
Epoch 28/50, Loss: 44.3464
Epoch 29/50, Loss: 43.8943
Epoch 30/50, Loss: 43.5106
Epoch 31/50, Loss: 43.2436
Epoch 32/50, Loss: 43.0186
Epoch 33/50, Loss: 42.6671
Epoch 34/50, Loss: 42.3923
Epoch 35/50, Loss: 42.2434
Epoch 36/50, Loss: 41.9478
Epoch 37/50, Loss: 41.6607
Epoch 38/

In [ ]:
# Torch datasets
train_loader = DataLoader(TensorDataset(torch.tensor(X_legit, dtype=torch.float32)), batch_size=2048, shuffle=True)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.int)

# Define the Autoencoder with BatchNorm
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Linear(32, 16),
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, input_dim),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Model setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Autoencoder(X.shape[1]).to(device)
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Train the model
EPOCHS = 100
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        x_batch = batch[0].to(device)
        optimizer.zero_grad()
        output = model(x_batch)
        loss = criterion(output, x_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss:.4f}")

# Compute threshold from reconstruction error of legit
model.eval()
with torch.no_grad():
    reconstructions = model(torch.tensor(X_legit, dtype=torch.float32).to(device))
    recon_mse = torch.mean((torch.tensor(X_legit) - reconstructions.cpu())**2, dim=1)
    threshold = recon_mse.mean() + 3 * recon_mse.std()

# Evaluate on test set
with torch.no_grad():
    test_recon = model(X_test_tensor.to(device))
    test_mse = torch.mean((X_test_tensor - test_recon.cpu()) ** 2, dim=1)
    y_pred = (test_mse > threshold).int()

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, digits=4, target_names=["Legit", "Fraud"]))

# Save model
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)
base_filename = "fraud_autoencoder_tuned"
i = 0
while True:
    filename = f"{base_filename}_{i:02d}.pt"
    filepath = os.path.join(model_dir, filename)
    if not os.path.exists(filepath):
        break
    i += 1

torch.save(model.state_dict(), filepath)
print(f"Model saved to {filepath}")

Epoch 1/100, Loss: 47.1421
Epoch 2/100, Loss: 39.2589
Epoch 3/100, Loss: 34.5119
Epoch 4/100, Loss: 30.9749
Epoch 5/100, Loss: 28.0556
Epoch 6/100, Loss: 25.6490
Epoch 7/100, Loss: 23.5387
Epoch 8/100, Loss: 21.7951
Epoch 9/100, Loss: 20.2685
Epoch 10/100, Loss: 18.9159
Epoch 11/100, Loss: 17.6562
Epoch 12/100, Loss: 16.4745
Epoch 13/100, Loss: 15.5342
Epoch 14/100, Loss: 14.7033
Epoch 15/100, Loss: 13.9862
Epoch 16/100, Loss: 13.3423
Epoch 17/100, Loss: 12.7732
Epoch 18/100, Loss: 12.2411
Epoch 19/100, Loss: 11.7592
Epoch 20/100, Loss: 11.3519
Epoch 21/100, Loss: 10.9794
Epoch 22/100, Loss: 10.6057
Epoch 23/100, Loss: 10.2707
Epoch 24/100, Loss: 9.9937
Epoch 25/100, Loss: 9.7550
Epoch 26/100, Loss: 9.5133
Epoch 27/100, Loss: 9.2729
Epoch 28/100, Loss: 9.0752
Epoch 29/100, Loss: 8.8957
Epoch 30/100, Loss: 8.6965
Epoch 31/100, Loss: 8.5113
Epoch 32/100, Loss: 8.3491
Epoch 33/100, Loss: 8.2064
Epoch 34/100, Loss: 8.0510
Epoch 35/100, Loss: 7.9175
Epoch 36/100, Loss: 7.7868
Epoch 37/100, 

### Architecture
- **Autoencoder** with 3 layers in both the encoder and decoder.
- **Batch Normalization** is applied after each layer.
- Loss function: `SmoothL1Loss` (less sensitive to outliers).
- Optimizer: `Adam`, learning rate = 0.0001
- Epochs: 100, Batch size: 2048

### Anomaly Detection
- Reconstruction error (MSE) is computed using legitimate data only.
- Threshold = mean + 3 * std of the reconstruction error.
- If the error > threshold → classified as fraud.

In [ ]:
# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Dataset & Dataloader
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=True)

# Define model
class FraudNN(nn.Module):
    def __init__(self, input_dim):
        super(FraudNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

model = FraudNN(X.shape[1])

# Training setup
criterion = nn.BCELoss(weight=torch.tensor([((y == 0).sum() / (y == 1).sum())], dtype=torch.float32))  # pos_weight
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
EPOCHS = 50
for epoch in range(EPOCHS):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X).squeeze()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{EPOCHS} Loss: {loss.item():.4f}")

# Evaluation
model.eval()
with torch.no_grad():
    y_pred_prob = model(X_test_tensor).squeeze().numpy()
    y_pred = (y_pred_prob >= 0.7).astype(int)

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, digits=4, target_names=["Legit", "Fraud"]))

Epoch 1/50 Loss: 0.5379
Epoch 2/50 Loss: 1.4563
Epoch 3/50 Loss: 0.2301
Epoch 4/50 Loss: 0.3584
Epoch 5/50 Loss: 2.2893
Epoch 6/50 Loss: 0.1856
Epoch 7/50 Loss: 0.2303
Epoch 8/50 Loss: 0.4676
Epoch 9/50 Loss: 0.0784
Epoch 10/50 Loss: 0.4658
Epoch 11/50 Loss: 0.1780
Epoch 12/50 Loss: 0.2349
Epoch 13/50 Loss: 0.5253
Epoch 14/50 Loss: 0.6885
Epoch 15/50 Loss: 0.1232
Epoch 16/50 Loss: 0.3987
Epoch 17/50 Loss: 0.2575
Epoch 18/50 Loss: 0.1901
Epoch 19/50 Loss: 0.1065
Epoch 20/50 Loss: 0.2203
Epoch 21/50 Loss: 0.3156
Epoch 22/50 Loss: 0.0985
Epoch 23/50 Loss: 0.0811
Epoch 24/50 Loss: 0.1368
Epoch 25/50 Loss: 0.6716
Epoch 26/50 Loss: 22.5842
Epoch 27/50 Loss: 0.5805
Epoch 28/50 Loss: 0.5041
Epoch 29/50 Loss: 0.2432
Epoch 30/50 Loss: 0.0981
Epoch 31/50 Loss: 0.9119
Epoch 32/50 Loss: 0.0530
Epoch 33/50 Loss: 0.8818
Epoch 34/50 Loss: 0.9893
Epoch 35/50 Loss: 0.5520
Epoch 36/50 Loss: 3.5237
Epoch 37/50 Loss: 0.0722
Epoch 38/50 Loss: 3.2508
Epoch 39/50 Loss: 0.1599
Epoch 40/50 Loss: 0.2779
Epoch 41

1. **Model Improvements**
   - Added two `Dropout(0.5)` layers (instead of 0.3) to reduce overfitting.
   - Used `BCEWithLogitsLoss` instead of `BCELoss`, which is numerically more stable and avoids applying `Sigmoid` inside the model.

2. **Class Imbalance Handling**
   - Passed `pos_weight` directly to `BCEWithLogitsLoss` based on the ratio of legit to fraud cases in the training set.

3. **Optimization**
   - Replaced the standard `Adam` optimizer with `AdamW`, which adds better weight decay regularization (`weight_decay=1e-5`).

4. **Batch Size**
   - Increased the batch size to **2048** for more stable gradient estimates.

5. **Early Stopping**
   - Implemented early stopping with a **patience of 10 epochs**, saving the best model based on training loss.

6. **Evaluation**
   - Applied `sigmoid` manually during evaluation.
   - Used a **custom threshold of 0.7** to reduce false positives.

7. **Model Saving**
   - Saved the final model weights as `fraud_nn_tuned.pt` in the `models/` directory.

In [ ]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=2048, shuffle=True)

# Define model
class FraudNN(nn.Module):
    def __init__(self, input_dim):
        super(FraudNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32, 1)  # no sigmoid!
        )

    def forward(self, x):
        return self.model(x)

model = FraudNN(input_dim=X_train.shape[1])

# Define loss with class imbalance
pos_weight = torch.tensor([(y_train == 0).sum() / (y_train == 1).sum()], dtype=torch.float32)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)

# Train with early stopping
EPOCHS = 100
patience = 10
min_loss = float('inf')
wait = 0

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{EPOCHS} Loss: {avg_loss:.4f}")

    # Early stopping check
    if avg_loss < min_loss:
        min_loss = avg_loss
        wait = 0
        best_model = model.state_dict()
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping triggered.")
            break

# Load best weights
model.load_state_dict(best_model)

# Evaluation
model.eval()
with torch.no_grad():
    y_logits = model(X_test_tensor)
    y_prob = torch.sigmoid(y_logits).numpy()
    y_pred = (y_prob >= 0.7).astype(int)

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, digits=4, target_names=["Legit", "Fraud"]))

# Save model
os.makedirs("models", exist_ok=True)
torch.save(model.state_dict(), "models/fraud_nn_tuned.pt")
print("Model saved to models/fraud_nn_tuned.pt")

Epoch 1/100 Loss: 0.9239
Epoch 2/100 Loss: 0.4933
Epoch 3/100 Loss: 0.4035
Epoch 4/100 Loss: 0.3976
Epoch 5/100 Loss: 0.3603
Epoch 6/100 Loss: 0.3368
Epoch 7/100 Loss: 0.3422
Epoch 8/100 Loss: 0.2868
Epoch 9/100 Loss: 0.3063
Epoch 10/100 Loss: 0.2801
Epoch 11/100 Loss: 0.3071
Epoch 12/100 Loss: 0.2841
Epoch 13/100 Loss: 0.2808
Epoch 14/100 Loss: 0.2680
Epoch 15/100 Loss: 0.2892
Epoch 16/100 Loss: 0.2571
Epoch 17/100 Loss: 0.2551
Epoch 18/100 Loss: 0.2266
Epoch 19/100 Loss: 0.2561
Epoch 20/100 Loss: 0.2241
Epoch 21/100 Loss: 0.2171
Epoch 22/100 Loss: 0.2303
Epoch 23/100 Loss: 0.2234
Epoch 24/100 Loss: 0.2134
Epoch 25/100 Loss: 0.2229
Epoch 26/100 Loss: 0.2277
Epoch 27/100 Loss: 0.2094
Epoch 28/100 Loss: 0.2020
Epoch 29/100 Loss: 0.1955
Epoch 30/100 Loss: 0.1803
Epoch 31/100 Loss: 0.1886
Epoch 32/100 Loss: 0.2216
Epoch 33/100 Loss: 0.1918
Epoch 34/100 Loss: 0.1587
Epoch 35/100 Loss: 0.1901
Epoch 36/100 Loss: 0.1689
Epoch 37/100 Loss: 0.1713
Epoch 38/100 Loss: 0.1532
Epoch 39/100 Loss: 0.

## Autoencoder for Anomaly Detection (PyTorch)

1. **Training on Legitimate Transactions Only**  
   - The Autoencoder was trained **only on legit transactions** (`Class == 0`) to learn a clean reconstruction pattern.

2. **Model Architecture**  
   - Encoder: `input_dim → 20 → 10`  
   - Decoder: `10 → 20 → input_dim`  
   - Activation: ReLU between layers.

3. **Training Setup**  
   - Optimizer: Adam with `lr=1e-3`  
   - Loss function: Mean Squared Error (MSE)  
   - Trained for **200 epochs** on mini-batches of size **2048**.

## Conclusions Neural Network (NN)

1. **General Performance**
   - The tuned neural network model achieved **very high accuracy (~99.97%)**, close to the best tree-based models.
   - However, as always, accuracy is **misleading** due to class imbalance.

2. **Recall vs. Precision Trade-off**
   - With the standard threshold, the model achieved **recall ≈0.79** and **precision ≈0.99** for fraud detection:
     - This means the model **rarely makes false alarms** but still **misses some fraud cases**.
   - When the threshold was lowered (e.g., 0.1), recall increased to **~0.89**, but precision dropped to **~0.19**:
     - The model detected more fraud but **made many incorrect fraud predictions**.

3. **F1-score and Balance**
   - F1-score ranged from **~0.32 to ~0.88**, depending on the threshold.
   - Indicates that the model can either:
     - Be **very conservative** (few false positives, lower recall), or
     - Be **aggressive** (high recall, low precision), depending on threshold tuning.

4. **Conclusion**
   - The neural network shows **potential**, but requires **careful threshold selection** depending on business priorities.
   - While it does not outperform the best tuned XGBoost or CatBoost models, its results are **still solid**.
   - May be useful in ensemble settings or where neural networks are preferred.